In [1]:
import pandas as pd
import joblib

In [2]:
data = pd.read_csv("act_labeling_data_ohencoding.csv",encoding='cp949')

In [3]:
data.head()

,CUSTNO,GNO,CBSCORE,CBSCOREGRD,CREDITOTAMT,YSALEAMT,ESTMM,ASSETAMT,IMSAAMT,IMJUAMT,...,KIND_JOB_K,KIND_JOB_L,KIND_JOB_M,KIND_JOB_N,KIND_JOB_O,KIND_JOB_P,KIND_JOB_Q,KIND_JOB_R,KIND_JOB_S,KIND_JOB_T
0,475821,l180202101898,945,1,20,2038,37,10,10,0,...,0,0,0,0,0,0,0,0,0,0
1,74417,l230201700120,894,2,19,1175,9,25,0,25,...,0,0,0,0,0,0,0,0,0,0
2,387787,l110201603233,906,1,0,763,185,10,10,0,...,0,0,0,0,0,0,0,0,0,0
3,395418,l230201700254,881,2,14,724,11,10,10,0,...,0,0,0,0,0,0,0,0,0,0
4,190372,l200201601418,725,6,7,335,167,80,2,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
features = data[['CBSCORE', 'CBSCOREGRD', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM', 'ASSETAMT', 'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT',  'LABORCNT', 'KOSPI', '환율',
       'GDP', '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률', '소비자심리지수',
       'KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T']]

act = data['ACTCD']

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler = scaler.fit(features)

In [13]:
##joblib.dump(scaler,'Robustscaler_save.save')

['Robustscaler_save.save']

In [6]:
scaler.transform(features)

array([[9.49290061e-01, 0.00000000e+00, 2.33808744e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.97565923e-01, 1.66666667e-01, 2.22118307e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.09736308e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [7.34279919e-01, 8.33333333e-01, 1.05213935e-03, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [9.52332657e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.99594320e-01, 1.66666667e-01, 5.84521861e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [9]:
from sklearn.model_selection import train_test_split


train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=42)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 43) (11736, 43) (27381,) (11736,)


In [10]:
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.over_sampling import BorderlineSMOTE

smtt = BorderlineSMOTE(random_state=42)

smt_train_x, smt_train_y = smtt.fit_resample(train_x,train_y)


In [11]:
from sklearn.linear_model import LogisticRegression

KNC = LogisticRegression(random_state=42,solver = 'saga')
KNC.fit(smt_train_x,smt_train_y)

LogisticRegression(random_state=42, solver='saga')

In [18]:
joblib.dump(KNC,"LogisticRegression.save")

['LogisticRegression.save']

In [12]:
pred_y = KNC.predict_proba(test_x)

In [13]:
column_names = ["Prob_Yes", "Prob_No"]



In [14]:
data_df = pd.DataFrame(pred_y,columns=column_names)

In [17]:
data_df['Prob_Yes'].value_counts()

0.355023    4
0.467595    4
0.415121    3
0.446613    3
0.336840    3
           ..
0.606449    1
0.424539    1
0.441356    1
0.620808    1
0.604141    1
Name: Prob_Yes, Length: 11546, dtype: int64

In [16]:
bbb=pred_y[:,0:2]

In [77]:
bbb

array([[1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       ...,
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.42857143, 0.57142857]])

In [78]:
test_x['prob_Yes'],test_x['prob_No']=bbb

ValueError: too many values to unpack (expected 2)

In [75]:
test_x

,CBSCORE,CBSCOREGRD,CREDITOTAMT,YSALEAMT,ESTMM,ASSETAMT,IMSAAMT,IMJUAMT,BUSAAMT,BUJUAMT,...,KIND_JOB_M,KIND_JOB_N,KIND_JOB_O,KIND_JOB_P,KIND_JOB_Q,KIND_JOB_R,KIND_JOB_S,KIND_JOB_T,prob_Yes,"(prob_Yes, prob_No)"
35620,933,1,1,0,2,46,46,0,0,0,...,0,0,0,0,0,0,0,0,1.000000,1.000000
13304,834,4,0,136,19,22,22,0,0,0,...,0,0,0,0,0,0,0,0,1.000000,1.000000
38803,824,4,10,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.000000,1.000000
8618,895,2,63,230,57,265,40,0,0,225,...,0,0,0,0,0,0,0,0,0.142857,0.142857
36794,830,4,10,0,5,277,7,0,0,270,...,0,0,0,0,0,0,0,0,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4064,824,4,79,450,143,190,30,0,0,160,...,0,0,0,0,0,0,0,0,1.000000,1.000000
7401,920,1,0,269,102,390,0,0,0,390,...,0,0,0,0,0,0,0,0,0.714286,0.714286
28336,871,3,4,20,136,20,20,0,0,0,...,0,0,0,0,0,0,1,0,1.000000,1.000000
34226,933,1,0,0,3,25,25,0,0,0,...,0,0,0,0,0,0,0,0,1.000000,1.000000


In [33]:
new = pd.concat([test_x,data_df],axis=1)

In [23]:
test_x['Prob_yes']=data_df['Prob_Yes']

C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [24]:
test_x

,CBSCORE,CBSCOREGRD,CREDITOTAMT,YSALEAMT,ESTMM,ASSETAMT,IMSAAMT,IMJUAMT,BUSAAMT,BUJUAMT,...,KIND_JOB_L,KIND_JOB_M,KIND_JOB_N,KIND_JOB_O,KIND_JOB_P,KIND_JOB_Q,KIND_JOB_R,KIND_JOB_S,KIND_JOB_T,Prob_yes
35620,933,1,1,0,2,46,46,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
13304,834,4,0,136,19,22,22,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
38803,824,4,10,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
8618,895,2,63,230,57,265,40,0,0,225,...,0,0,0,0,0,0,0,0,0,1.000000
36794,830,4,10,0,5,277,7,0,0,270,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4064,824,4,79,450,143,190,30,0,0,160,...,0,0,0,0,0,0,0,0,0,0.571429
7401,920,1,0,269,102,390,0,0,0,390,...,0,0,0,0,0,0,0,0,0,1.000000
28336,871,3,4,20,136,20,20,0,0,0,...,0,0,0,0,0,0,0,1,0,NaN
34226,933,1,0,0,3,25,25,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
